In [ ]:
import math
import datetime
import pandas as pd
import pandas.io 
import pandas_datareader as pdr
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from tkinter import *
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
    


#variables for the instances
window=30
windows=[30, 60, 90, 120]
quantiles=[0.25, 0.75]
bins=100
normed=True
bench='^GSPC'
open=False
    
    
  
###Import Historical CSV Data
def get_data(data, start, end):
    

    high_price = data['High']
    low_price = data['Low']
    open_price = data['Open']
    close_price = data['Close']
    adj_close_price = data['Adj Close']

    adj_factor = adj_close_price / close_price


    adj_high_price = data['High'] * adj_factor
    adj_low_price = data['Low'] * adj_factor
    adj_open_price = data['Open'] * adj_factor
    
    returns = (data['Adj Close'] / data['Adj Close'].shift(1)) - 1
    
    return pandas.DataFrame({
    'Open': data['Open'],
    'High': data['High'], 
    'Low': data['Low'],
    'Close': data['Close'],
    'Volume': data['Volume'],
    'Adj Open': adj_open_price,
    'Adj High': adj_high_price,
    'Adj Low': adj_low_price,
    'Adj Close': data['Adj Close'],
    'Return': returns
    }, index=data.index)




###Run Parkinson Equation for Implied Volatility
def parkinson_fn(prices, window=30, clean=True):
    
    num_of_days = 85
    adj_high = prices['Adj High']
    adj_low =  prices['Adj Low']
    mult_factor = 1 / (4 * math.log(2))
    daily_variance = (adj_high / adj_low).apply(np.log)

    adj_daily_variance = mult_factor * (daily_variance)**2
    
    def total_realized_vol(variance):
        return math.sqrt(num_of_days **  variance.mean())
    
    result = adj_daily_variance.rolling(window).apply(total_realized_vol)
    result[:window-1] = np.nan
    
    if clean:
        return result.dropna()
    else:
        return result


                  
### Initialize Input Variables 
btc_historical_data = pandas.read_csv('BTC_USDv1.csv')

start = dt.datetime(2022, 5, 5)
end = dt.datetime(2022, 6, 9)

                
prices = get__data(btc_historical_data, start, end)
realized_vol = parkinson_fn(prices, window=30, clean=True)

#mean = realized_vol.rolling(window).mean()
#std = realized_vol.rolling(window).std()

prices['Historical Volatility'] = realized_vol
print(prices)